In [1]:
!pip install monsterapi

In [4]:
import os
import time
from monsterapi import client as m_client
from google.colab import output
from base64 import b64decode
from IPython.display import Audio, display, Javascript, Image

# --- CONFIGURATION ---
# Paste your MonsterAPI Key here
MONSTER_API_KEY = "YOUR_MONSTER_API_KEY"
monster_client = m_client(api_key=MONSTER_API_KEY)

# --- JS Code for Microphone Access ---
RECORD_JS = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    reader = new FileReader()
    reader.readAsDataURL(blob)
    reader.onloadend = () => resolve(reader.result)
  }
  recorder.stop()
})
"""

def speech_to_image():
    # 1. RECORD AUDIO
    display(Javascript(RECORD_JS))
    print("Speak your image prompt (Any language)...")
    s = output.eval_js('record(5000)') # Records for 5 seconds
    b = b64decode(s.split(',')[1])

    with open('input_audio.wav', 'wb') as f:
        f.write(b)

    # 2. TRANSCRIBE (Speech to Text)
    print("Transcribing with Whisper...")
    transcription_task = monster_client.get_response("whisper", {
        "file": "input_audio.wav" # Corrected: Pass the filename string
    })

    # Note: MonsterAPI tasks are asynchronous. For Whisper, it usually returns text immediately.
    prompt = transcription_task.get("text", "")
    print(f"Detected Prompt: {prompt}")

    if not prompt:
        print("Could not detect any speech. Try again!")
        return

    # 3. GENERATE IMAGE (Text to Image)
    print("Generating image with SDXL...")
    image_payload = {
        "prompt": prompt,
        "negprompt": "blurry, low quality, distorted",
        "samples": 1,
        "aspect_ratio": "square"
    }

    # Launch the task
    task_response = monster_client.get_response("sdxl-base", image_payload)
    process_id = task_response.get("process_id")

    # Wait for the image to be generated
    print("Waiting for image...")
    while True:
        status_response = monster_client.wait_and_get_result(process_id)
        if "result" in status_response:
            image_url = status_response["result"]["output"][0]
            display(Image(url=image_url))
            print(f"Image ready! URL: {image_url}")
            break
        time.sleep(2)

# Run the system
speech_to_image()

<IPython.core.display.Javascript object>

Speak your image prompt (Any language)...
Transcribing with Whisper...


ConnectionError: HTTPSConnectionPool(host='api.monsterapi.ai', port=443): Max retries exceeded with url: /v1/generate/whisper (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x78c5a287eb10>: Failed to resolve 'api.monsterapi.ai' ([Errno -2] Name or service not known)"))